In [13]:
# main.ipynb
# Archivo base en el que se maneja la experimentación del diseño metodológico.
# Objetivo general: Utilizar los datos publicos (ubicados en la carpeta /data/)
#     de municipios en los que existe información para diseñar una estrategia
#     para distribuir alimentos en el territorio nacional.
# Objetivos específicos:

#     ---pronostico_poblacional---
#     [IA]1. Determinar por municipio que herramienta es mejor para pronosticar
#            la población en 10 años a partir del año actual.
#               a. Regresión Lineal Multiple || Multiple Linear Regression.
#               b. Arboles de Regresión || Regression Tree.
#               c. Máquinas de Vectores de Soporte || Support Vector Machine.
#               d. Bosques Aleatorios || Random Forest Regression.
#               e. Redes neurales || Deep Learning.
#               f. Regresión Tradicional.
#     ---capacidad_y_costo---
#     [IO]2. Determinar las capacidades y costos de almacenamiento de alimentos
#            en cada municipio utilizando los datos del pronóstico, los datos
#            abiertos y consideraciones de los analístas.
#     ---pronosticar_capacidad_y_costo---
#     [IA]3. Entrenar un clasificador desde los datos de capacidades y costos
#            de almacenamiento por municipio.
#               a. Arboles de Decision || Decision Tree.
#               b. Análisis Discriminante Lineal || Linear Discriminant Analysis.
#               c. Regresión Logística || Logistic Regression.
#               d. Máquinas de Vectores de Soporte || Support Vector Machine.
#               e. Redes Neurales || Deep Learning.
#               f. Análisis de frecuencias. [Aún está en desarrollo la idea]
#     ---cantidad_de_clusteres---
#     [IO]4. Proponer una cantidad de clusteres que permitan disminuir el costo
#            computacional del algoritmo optimizador y compararlo con los métodos
#            tradicionales.
#     ---generar_clusteres---
#     [IA]5. Dividir los municipios en clústeres, rectificando que es viable
#            satisfacer la demanda de alimentos con la capacidad instalada del
#            municipio.
#               a. k-means.
#               b. Mapa Autoorganizado || Self-Organizing Map.
#               c. Agrupamiento Jerárquico || Agglomerative Clustering.
#               d. DBSCAN.
#     ---solucionar_cflp---
#     [IO]6. Resolver el cflp para diferentes escenarios.
#               a. Solución ingenua (todos los municipios tienen un centro de
#                  distribución).
#               b. Datos completos sin clusterizar.
#               c. Dividido por clústers.
#


In [14]:
# Manejo básico de archivos
import os, sys, warnings

# Registro del tiempo
import time

# Manejo de datos
import pandas as pd
import numpy as np
import pulp as pulp

# Funciones personalizadas de /funciones/funciones.py, /funciones/alistamiento.py, /funciones/miscelaneas.py
from funciones.funciones import *
from funciones.alistamiento import *
from funciones.miscelaneas import *

# funciones del flujo de trabajo
from funciones.pronostico_poblacional import *
from funciones.capacidad_y_costo import *
from funciones.predecir_capacidad_y_costo import *
from funciones.cantidad_de_clusteres import *
from funciones.generacion_de_clusteres import *


In [15]:
# Funciones



def actualizar_resutados_ingenuos(resultados, key, ingenua):
    resultados["tipo_de_datos"].append(key)
    resultados["tipo_de_solucion"].append("ingenua")
    resultados["algoritmo_de_clusterizacion"].append("ninguno")
    resultados["costo_total"].append(ingenua[0])
    resultados["cantidad_de_centros_de_distribucion"].append(ingenua[1])
    resultados["tiempo_de_ejecucion"].append(ingenua[2])
    resultados["suma_de_demanda_por_cluster"].append(ingenua[3])
    resultados["suma_de_demanda_satisfecha_por_cluster"].append(ingenua[4])
    resultados["suma_de_capacidad_por_cluster"].append(ingenua[5])
    resultados["suma_de_capacidad_utilizada_por_cluster"].append(ingenua[6])
    resultados["estado"].append(
        "satisfecho" if ingenua[3] <= ingenua[5] else "insatisfecho"
    )
    return resultados


def leer_datos_solucion_cflp(comida_per_capita):
    datos_crudos = {
        "datos_completos": {
            "poblacion": pd.read_csv(
                "resultados/tablas/pronostico_poblacional/datos_completos.csv",
                index_col=0,
            )["Poblacion_2034"],
            "demanda": pd.read_csv(
                "resultados/tablas/pronostico_poblacional/datos_completos.csv",
                index_col=0,
            )["Poblacion_2034"].rename("demanda")
            * comida_per_capita
            * 7,  # 7 días de comida
            "lat": pd.read_csv("data/datos_completos/municipios.csv", index_col=0)[
                "lat"
            ],
            "lon": pd.read_csv("data/datos_completos/municipios.csv", index_col=0)[
                "lon"
            ],
            "capacidad": pd.read_csv(
                "resultados/tablas/capacidad_y_costo/demanda_completa.csv", index_col=0
            )["capacidad"],
            "precio": pd.read_csv(
                "resultados/tablas/capacidad_y_costo/demanda_completa.csv", index_col=0
            )["precio"],
            "kmeans": pd.read_csv(
                "resultados/tablas/clusteres/datos_completos.csv", index_col=0
            )["kmeans"],
            "som": pd.read_csv(
                "resultados/tablas/clusteres/datos_completos.csv", index_col=0
            )["som"],
            "agglomerative": pd.read_csv(
                "resultados/tablas/clusteres/datos_completos.csv", index_col=0
            )["agglomerative"],
            "dbscan": pd.read_csv(
                "resultados/tablas/clusteres/datos_completos.csv", index_col=0
            )["dbscan"],
        },
        "datos_imperfectos": {
            "poblacion": pd.read_csv(
                "resultados/tablas/pronostico_poblacional/datos_imperfectos.csv",
                index_col=0,
            )["Poblacion_2034"],
            "demanda": pd.read_csv(
                "resultados/tablas/pronostico_poblacional/datos_imperfectos.csv",
                index_col=0,
            )["Poblacion_2034"].rename("demanda")
            * comida_per_capita
            * 7,  # 7 días de comida
            "lat": pd.read_csv("data/datos_imperfectos/municipios.csv", index_col=0)[
                "lat"
            ],
            "lon": pd.read_csv("data/datos_imperfectos/municipios.csv", index_col=0)[
                "lon"
            ],
            "capacidad": pd.read_csv(
                "resultados/tablas/capacidad_y_costo/demanda_imperfecta.csv",
                index_col=0,
            )["capacidad"],
            "precio": pd.read_csv(
                "resultados/tablas/capacidad_y_costo/demanda_imperfecta.csv",
                index_col=0,
            )["precio"],
            "kmeans": pd.read_csv(
                "resultados/tablas/clusteres/datos_imperfectos.csv", index_col=0
            )["kmeans"],
            "som": pd.read_csv(
                "resultados/tablas/clusteres/datos_imperfectos.csv", index_col=0
            )["som"],
            "agglomerative": pd.read_csv(
                "resultados/tablas/clusteres/datos_imperfectos.csv", index_col=0
            )["agglomerative"],
            "dbscan": pd.read_csv(
                "resultados/tablas/clusteres/datos_imperfectos.csv", index_col=0
            )["dbscan"],
        },
    }
    matriz_de_costos = {
        "datos_completos": pd.read_csv(
            "data/datos_completos/matriz-de-costos.csv", index_col=0
        ),
        "datos_imperfectos": pd.read_csv(
            "data/datos_imperfectos/matriz-de-costos.csv", index_col=0
        ),
    }
    # unir los datos de los municipios en dataframes
    datos = {}
    for tipo_de_datos, datos_por_tipo in datos_crudos.items():
        datos[tipo_de_datos] = pd.concat(
            [
                datos_por_tipo["poblacion"],
                datos_por_tipo["demanda"],
                datos_por_tipo["lat"],
                datos_por_tipo["lon"],
                datos_por_tipo["capacidad"],
                datos_por_tipo["precio"],
                datos_por_tipo["kmeans"],
                datos_por_tipo["som"],
                datos_por_tipo["agglomerative"],
                datos_por_tipo["dbscan"],
            ],
            axis=1,
        )
        datos[tipo_de_datos].index.name = "divipola"
        datos[tipo_de_datos].index = datos[tipo_de_datos].index.astype(int)
        matriz_de_costos[tipo_de_datos].columns = matriz_de_costos[
            tipo_de_datos
        ].columns.astype(int)
        matriz_de_costos[tipo_de_datos].index = matriz_de_costos[
            tipo_de_datos
        ].index.astype(int)
    return datos, matriz_de_costos


def solucion_ingenua(
    datos, key
) -> tuple[float, int, float, float, float, float, float]:
    """
    Función para que utiliza la solución ingenua para resolver el problema de
    la facilidad de localización de centros de distribución capacitados para
    satisfacer la demanda de alimentos en los municipios.

    La función ingenua es aquella que asigna un centro de distribución por
    municipio.

    adicionalmente se crea un archivo de excel con los resultados de cada variable:
    i: Lista de potenciales centros de distribución
    j: Lista de municipios a los que se les asigna un centro de distribución
    Y_i: si un centro de distribución fue asignado o no {1, 0}
    X_ij: La cantidad en toneladas que satisface cada centro de distribución por municipio
    """
    tiempo_inicial = time.time()
    # el costo total es activar todos los lugares
    costo = sum(datos["precio"])
    cantidad_cd = len(datos)
    suma_de_demanda = sum(datos["demanda"])
    suma_de_demanda_por_cluster = suma_de_demanda
    suma_de_capacidad = sum(datos["capacidad"])
    suma_de_capacidad_utilizada = suma_de_demanda
    tiempo_final = time.time() - tiempo_inicial
    if suma_de_demanda > suma_de_capacidad:
        print("        No se puede satisfacer la demanda con la capacidad instalada")

    # crear el archivo de excel si no existe
    if not os.path.exists(
        f"resultados/tablas/solucionar_cflp/soluciones/{key}-ingenua.xlsx"
    ):
        print("        Creando archivo de excel", end="\r")
        # y vector de 1s
        y = [1] * len(datos)
        # x matriz de cantidad_cd x cantidad_cd con la diagonal igual a la demanda
        x = np.zeros((len(datos), len(datos)))
        np.fill_diagonal(x, datos["demanda"])
        # crear el dataframe
        df_x = pd.DataFrame(x, index=datos.index, columns=datos.index)
        df_y = pd.DataFrame(y, index=datos.index, columns=["Y"])
        # añadir una columna de 1s a y con el nombre de 'cluster'
        df_y["cluster"] = 1
        # añadir una columna con el nombre del 'municipio' a y
        df_y["municipio"] = pd.read_csv(f"data/{key}/municipios.csv", index_col=0).loc[
            datos.index, "municipio"
        ]
        with pd.ExcelWriter(
            f"resultados/tablas/solucionar_cflp/soluciones/{key}-ingenua.xlsx"
        ) as writer:
            df_x.to_excel(writer, sheet_name="X")
            df_y.to_excel(writer, sheet_name="Y")
        print("        Archivo de excel creado satisfactoriamente")
    return (
        costo,
        cantidad_cd,
        tiempo_final,
        suma_de_demanda,
        suma_de_demanda_por_cluster,
        suma_de_capacidad,
        suma_de_capacidad_utilizada,
    )


In [16]:
# PARAMETROS

from scipy import cluster


comida_per_capita = 0.00087617  # 0.87617 kg por persona / día
densidad_de_alimentos = 537 / 1000  # 537 kg por m3 (5 metros de altura)
RANDOM_SEED = 11

crear_estructura_de_archivos()
procesar_datos_completos(comida_per_capita, densidad_de_alimentos)
procesar_datos_imperfectos()

pronostico_poblacional(RANDOM_SEED)
capacidad_y_costo(comida_per_capita)
cantidad_de_clusteres(RANDOM_SEED)
generar_clusteres(RANDOM_SEED, n_cluster=6)


def solucionar_cflp(comida_per_capita):
    """
    Función para solucionar el problema de la facilidad de localización de
    centros de distribución capacitados para satisfacer la demanda de
    alimentos en los municipios.

    Pseudocódigo:
    para cada tipo de datos [datos_completos, datos_imperfectos]

    1. Leer los datos necesarios.
        - Datos de los municipios.
            - divipola (primera columna de todos los archivos)
            - población (resultados/tablas/pronostico_poblacional/{datos_completos, datos_imperfectos}.csv) - columna ['Poblacion_2034']
            - lat y lon (data/{datos_completos, datos_imperfectos}/municipios.csv) - columnas ['lat', 'lon'] o (resultados/tablas/clusteres/{datos_completos, datos_imperfectos}.csv) - columnas ['lat', 'lon']
            - capacidad de almacenamiento  (resultados/tablas/capacidad_y_costo/{demanda_completa, demanda_imperfecta}.csv) - columna ['capacidad']
            - costo de almacenamiento (resultados/tablas/capacidad_y_costo/{demanda_completa, demanda_imperfecta}.csv) - columna ['precio']
            - cluster al que pertenece (resultados/tablas/clusteres/{datos_completos, datos_imperfectos}.csv) - columnas ['kmeans', 'som', 'agglomerative', 'dbscan']
        - Datos de costos de transporte. (data/{datos_completos, datos_imperfectos}/matriz-de-costos.csv)
            - divipola origen (primera columna de todos los archivos)
            - divipola destino (primera fila de todos los archivos [encabezado])
            - costo de transporte (resto de la matriz)
        # revisar que columnas e indices queden como enteros
    [Poblacion_2034	demanda	lat	lon	capacidad	precio	kmeans	som	agglomerative	dbscan]
    2. Ordenar y estructurar los datos [crear la demanda=poblacion*comida_per_capita*7].
    3. Crear diccionario que recolectará los resultados.
        - tipo de datos [datos_completos, datos_imperfectos]
        - tipo de solución [ingenua, sin clusterizar, clusterizada]
        - algoritmo de clusterización [k-means, som, agglomerative, dbscan]
        - costo total
        - cantidad de centros de distribución
        - tiempo de ejecución
        - suma de la demanda por cluster
        - suma de demanda satisfecha por cluster
        - suma de la capacidad por cluster
        - suma de la capacidad utilizada por cluster
    4. Resolver el problema para cada tipo de solución.
        - Solución ingenua.
        - Solución sin clusterizar.
        - Solución clusterizada.
    5. Guardar los resultados en un archivo csv.
    """
    datos, matriz_de_costos = leer_datos_solucion_cflp(comida_per_capita)
    resultados = crear_diccionario_de_resultados()
    for key, value in datos.items():
        costos = matriz_de_costos[key]
        #### Solución ingenua ####
        print(f"Resolviendo el problema para {key}")
        # Solución ingenua
        print("    Procesando solución ingenua", end="\r")
        ingenua = solucion_ingenua(value, key)
        resultados = actualizar_resutados_ingenuos(resultados, key, ingenua)
        print("    Solución ingenua procesada satisfactoriamente")
        
        #### Solución sin clusterizar ####
        sin_clusterizar = solucion_sin_clusterizar_MC(value, costos, key+"-sin-clusterizar", cluster_name=1)
        

    display(pd.DataFrame(resultados))

def crear_diccionario_de_resultados():
    resultados = {
        "tipo_de_datos": [],
        "tipo_de_solucion": [],
        "algoritmo_de_clusterizacion": [],
        "costo_total": [],
        "cantidad_de_centros_de_distribucion": [],
        "tiempo_de_ejecucion": [],
        "estado": [],
        "suma_de_demanda_por_cluster": [],
        "suma_de_demanda_satisfecha_por_cluster": [],
        "suma_de_capacidad_por_cluster": [],
        "suma_de_capacidad_utilizada_por_cluster": [],
    }

    return resultados

def solucion_sin_clusterizar_MC(datos, costos, key, cluster_name):
    """
    Función que resuelve cflp, crea el archivo de excel con los resultados y
    retorna los resultados de la solución sin clusterizar.

    formulación:
    SETS:
        i: Lista de potenciales centros de distribución
        j: Lista de municipios a que demandan alimentos
    VARIABLES:
        Y_i: si un centro de distribución fue asignado o no {1, 0}
        X_ij: La cantidad en toneladas que se transportan de i a j
    PARAMETROS:
        c[i, j]: costo de transportar una tonelada de alimentos de i a j
        f[i]: costo de activar un centro de distribución
        a[i]: capacidad de almacenamiento de i
        b[j]: demanda de alimentos de j
    OBJETIVO:
        Minimizar el costo total
            min F = sum_{i in I} sum_{j in J} c[i, j] * X_ij + sum_{i in I} f[i] * Y_i
    RESTRICCIONES:
        1. sum_{i in I} X_ij = b[j] for all j in J
        2. X_ij <= a[i] * Y_i for all i in I, j in J
        3. X_ij >= 0 for all i in I, j in J
        4. Y_i = {0, 1} for all i in I

    El archivo de excel se crea si no existe un archivo con el nombre
    'resultados/tablas/solucionar_cflp/soluciones/{nombre}.xlsx'
    es un archivo con dos hojas:
        - X: con la variable X_ij
        - Y: con la variable Y_i + cluster_name + municipio



    
    """

solucionar_cflp(comida_per_capita)


Procesando los DATOS COMPLETOS
Los datos ya fueron procesados
Procesando los DATOS IMPERFECTOS
Los datos ya fueron procesados

El proceso de pronóstico poblacional ya fue realizado

La capacidad y costos ya fueron calculados

La propuesta de k para los datos ya fue generada.

Las tablas de clusteres ya han sido generadas.
Resolviendo el problema para datos_completos
    Solución ingenua procesada satisfactoriamente
Resolviendo el problema para datos_imperfectos
    Solución ingenua procesada satisfactoriamente


,tipo_de_datos,tipo_de_solucion,algoritmo_de_clusterizacion,costo_total,cantidad_de_centros_de_distribucion,tiempo_de_ejecucion,estado,suma_de_demanda_por_cluster,suma_de_demanda_satisfecha_por_cluster,suma_de_capacidad_por_cluster,suma_de_capacidad_utilizada_por_cluster
0,datos_completos,ingenua,ninguno,1.657508e+09,432,0.000391,satisfecho,252457.771590,252457.771590,822706.23,252457.771590
1,datos_imperfectos,ingenua,ninguno,3.778126e+09,1117,0.000465,satisfecho,295405.890101,295405.890101,1554370.23,295405.890101
